# Eager Execution

In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()

## Tensors
Basic tensor operations

In [2]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square([5, 6]))
print(tf.reduce_sum([[1,2, 3], [4, 5, 6]], axis=1))
print(tf.reduce_sum([[1,2, 3], [4, 5, 6]], axis=0))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)
tf.Tensor([ 6 15], shape=(2,), dtype=int32)
tf.Tensor([5 7 9], shape=(3,), dtype=int32)


In [3]:
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)
print(x)

(1, 2)
<dtype: 'int32'>
tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)


Tensors are immutable


In [4]:
# Convert Numpy arrays to tensor
x = np.array([1,2])
print(tf.convert_to_tensor(x))

# TensorFlow operation on numpy array returns tensor
x = np.ones([3, 3])

tensor = tf.multiply(x, 10)
print(tensor)

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor(
[[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]], shape=(3, 3), dtype=float64)


In [5]:
# Numpy operation on tensor returns numpy array
np.reshape(tensor, newshape=(-1,1))

array([[10.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.],
       [10.]])

## GPU Acceleration

In [6]:
x = tf.random_uniform([3, 3])

print("Is GPU available:")
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:")
print(tf.test.is_gpu_available('GPU:0'))

Is GPU available:
True
Is the Tensor on GPU #0:
True


In [7]:
def time_matmul(x):
  %timeit tf.matmul(x, x)

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)

# Force execution on GPU #0 if available
if tf.test.is_gpu_available():
    print("On GPU:")
    with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
        x = tf.random_uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU:
40.8 ms ± 2.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
On GPU:
82 µs ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Datasets

In [8]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3])

for i in ds_tensors:
    print(i)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


In [9]:
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
Line 2
Line 3""")
    
ds_file = tf.data.TextLineDataset(filename)

for i in ds_file:
    print(i)

tf.Tensor(b'Line 1', shape=(), dtype=string)
tf.Tensor(b'Line 2', shape=(), dtype=string)
tf.Tensor(b'Line 3', shape=(), dtype=string)


Iteration only works in eager execution mode

## Computing gradients

In [71]:
w = tf.Variable([[1.0, 2.0, 3.0]])

with tf.GradientTape() as tape:
    loss = w * w

# grad = 2*w
grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[2. 4. 6.]], shape=(1, 3), dtype=float32)


In [58]:
w = tf.Variable([[1.0, 2.0, 3.0]])

with tf.GradientTape() as tape:
    x = 3*w*w + 2*w
    loss = tf.log(x)

# Gradient d_loss/dw
grad_loss = tape.gradient(loss, w)
print(grad_loss)

tf.Tensor([[1.6       0.875     0.6060606]], shape=(1, 3), dtype=float32)


None


In [57]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
  return input * weight + bias

# A loss function using mean-squared error
def loss(weights, biases):
  error = prediction(training_inputs, weights, biases) - training_outputs
  return tf.reduce_mean(tf.square(error))

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
  with tf.GradientTape() as tape:
    loss_value = loss(weights, biases)
  return tape.gradient(loss_value, [weights, biases])

train_steps = 200
learning_rate = 0.01
# Start with arbitrary values for W and B on the same batch of data
W = tf.Variable(5.)
B = tf.Variable(10.)

print("Initial loss: {:.3f}".format(loss(W, B)))

for i in range(train_steps):
  dW, dB = grad(W, B)
  W.assign_sub(dW * learning_rate)
  B.assign_sub(dB * learning_rate)
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final loss: {:.3f}".format(loss(W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))

Initial loss: 68.646
Loss at step 000: 66.005
Loss at step 020: 30.314
Loss at step 040: 14.220
Loss at step 060: 6.958
Loss at step 080: 3.679
Loss at step 100: 2.198
Loss at step 120: 1.529
Loss at step 140: 1.227
Loss at step 160: 1.090
Loss at step 180: 1.028
Final loss: 1.001
W = 3.008985757827759, B = 2.113598585128784
